In [1]:
from scispacy.linking import EntityLinker
from os.path import join
from tqdm.notebook import tqdm
import pandas as pd
import gc
DATA_SOURCE = 'all_triples'
UMLS_Sem_Net_DIR = '/home/pc/Desktop/AdilStuff/Projects/SemRepMed/umls semantic network'
META_DIR = '/home/pc/Downloads/umls-2022AA-full/2022AA-full'
# all_data_entities.csv

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [5]:
def collect_garbage():
    print(gc.get_count())
    gc.collect()
    print(gc.get_count())


In [6]:
# collect_garbage()

## Entity Description Using Scispacy Linkage

In [2]:
from scispacy.umls_linking import UmlsEntityLinker

In [3]:
linker = UmlsEntityLinker(resolve_abbreviations=True, filter_for_definitions=False)

/home/pc/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pc/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
#dir(linker.umls)

In [15]:
def canonize_cui(cui):
    cacon_form = 'C0000000'
    if len(cui)<8:
        cui = cacon_form[0:-len(cui)]+cui
    elif 'C' not in  cui:
        cui = 'C' + cui
    return cui

def process_mult_cui(cui):
    origin_cui = None
    all_cuis = []
    for cu in cui.split('|'):
        if origin_cui is None:
            cu = canonize_cui(cu)
            origin_cui = cu
        else:
#             cu = origin_cui[0:len(origin_cui)-len(cu)] + cu
            if len(origin_cui)>=8:
                cu = origin_cui[0:-len(cu)] + cu
            else:
                cu = canonize_cui(cu)
                origin_cui = cu
        if len(origin_cui)>=8:
            all_cuis.append(cu)
    return all_cuis

def process_cui(cui):
    if '|' in cui:
        cuis = process_mult_cui(cui)
    else:
        cuis = [canonize_cui(cui)]
    return cuis

In [11]:
process_mult_cui('C0081938|10678|146712')

['C0081938', 'C0010678', 'C0146712']

In [12]:
process_mult_cui('3842|7214')

['C0003842', 'C0007214']

In [13]:
process_mult_cui(cui):

SyntaxError: invalid syntax (368372621.py, line 1)

In [16]:
def get_entity_descriptions(linker):
    df = pd.read_csv(join(DATA_SOURCE, 'all_data_entities_can.csv'), compression = 'gzip')
    all_descriptions = []
    pbar = tqdm(total = df.shape[0])
    for i, rec in df.iterrows():
        
        cuis = rec['ENTITY']
        sem_type = rec['SEMTYPE']
#         if '|' in cui:
#             cui = cui.split('|')[0]
        cuis = process_cui(cuis)
        for cui in cuis:
            ent_description = {}
            if cui in linker.umls.cui_to_entity:
                entity_data = linker.umls.cui_to_entity[cui]
                ent_description['ENTITY'] = cui
                ent_description['DEFINITION'] = entity_data.definition
                ent_description['ALIASES'] = '|'.join(entity_data.aliases)
                ent_description['NAME'] = entity_data.canonical_name
                all_descriptions.append(ent_description)
            else:
                print(cui)
                ent_description['ENTITY'] = cui
                ent_description['DEFINITION'] = None
                ent_description['ALIASES'] = None
                ent_description['NAME'] = None
        pbar.update()
    return pd.DataFrame(all_descriptions)

In [ ]:
C1413172|595

In [ ]:
3429|5715|10534|10671|51014

In [ ]:
C0003429

In [141]:
linker.umls.cui_to_entity['C1008626']

CUI: C1008626, Name: Galleriinae
Definition: None
TUI(s): T204
Aliases: (total: 0): 
	 

In [128]:
all_cuis = list(linker.umls.cui_to_entity.keys())

In [18]:
df = pd.read_csv(join(DATA_SOURCE, 'all_data_entities_can.csv'), compression = 'gzip')

In [19]:
df

,SEMTYPE,NAME,ORIGIN_ID,ENTITY
0,virs,Arboviruses,0,C0003725
1,virs,Tahyna virus,1,C0039258
2,virs,Eyach virus,2,C0318627
3,virs,California Group Viruses,3,C0446169
4,dsyn,Disease,4,C0012634
...,...,...,...,...
601503,popg,Filipino Americans,507379,C0597918
601504,irda,ioxaglate,507380,C0022038
601505,mnob,Flashlamp-pumped pulsed dye laser device,507381,C1289860
601506,fndg,Centralized myonuclei,507382,C1866020


In [20]:
ent_descriptions = get_entity_descriptions(linker)

  0%|          | 0/601508 [00:00<?, ?it/s]

C0446169
C0007523
C0718782
C0718782
C0021311
C1338204
C1338204
C0753156
C0753156
C1170884
C0403425
C1261469
C1302475
C1302475
C0382247
C0382247
C0008284
C0129831
C0129831
C0424419
C1365599
C1365599
C0027347
C1166614
C1457192
C1457192
C1182607
C0002649
C0006495
C0081029
C0375243
C0375243
C0020114
C0164207
C0164207
C0051072
C0051072
C1351761
C1351761
C1338065
C1338065
C0003347
C1355824
C1355824
C1355824
C1552516
C0131450
C0175677
C0871461
C0811398
C0811398
C0000185
C1367174
C1367174
C0301521
C0008434
C0022796
C0022796
C0064374
C0064374
C0001545
C0678903
C0082353
C0082353
C0082353
C0082353
C1512655
C0020114
C1336241
C1336241
C0293952
C0293952
C0003700
C1419252
C1419252
C1331386
C1331386
C100616496
C100616496
C1258430
C1336014
C1384115
C0034618
C0687732
C0298737
C0298737
C0040711
C0084153
C0084153
C0084153
C0084153
C0136319
C0136319
C0540602
C0679466
C0215970
C0215970
C0679670
C0006574
C0298472
C0298472
C0238806
C0023028
C1146872
C1146872
C0004734
C0000745
C0007070
C0026986
C0867388
C15109

C0010716
C1331559
C1331559
C110006328
C110006328
C110006328
C110006328
C110006328
C1335074
C1335074
C0005300
C0964859
C0010407
C0010407
C0691019
C0691019
C0296603
C1532623
C1532623
C1332545
C1332545
C1533434
C0001958
C0972810
C0972810
C0972810
C0972810
C0974193
C0974193
C0129984
C0129984
C0129984
C0031956
C0031956
C1366774
C1366774
C0164207
C0164207
C1336667
C1336667
C0676366
C0676366
C0034367
C0034367
C1411050
C1411050
C1337516
C1337516
C1337517
C1337517
C1310594
C1310594
C1310594
C0027292
C0027292
C1156146
C0901365
C0901365
C1364311
C1364311
C0150100
C1339044
C1339044
C0005079
C0858246
C1361725
C1361725
C1516370
C0036962
C0036962
C0004191
C0808143
C1523840
C0379857
C0379857
C0586173
C1516652
C0003759
C0989796
C0989796
C0008242
C0009360
C0003043
C0000199
C0422792
C0521393
C0001855
C0755155
C0755155
C1411131
C1411131
C0006046
C0051191
C0051191
C0489872
C1321974
C0070574
C0859917
C0023064
C0005621
C0029435
C0020040
C0000166
C1168022
C0257677
C1355814
C1355814
C0005879
C0174595
C0174595


C0094025
C0549419
C0057805
C0074609
C0074609
C0458360
C1134988
C1134988
C1134988
C1134988
C1046877
C0004553
C0004553
C1322800
C0667054
C0667054
C1142270
C0249760
C0249760
C1367914
C1367914
C1367132
C1367132
C0001553
C0022933
C1411984
C1411984
C1411984
C0001147
C0264951
C0771786
C0771786
C0009138
C1164656
C1164656
C0005542
C0105613
C0105613
C0006187
C0255135
C0255135
C0001353
C0006431
C0003491
C1415530
C1415530
C0006336
C1338091
C1338091
C1336850
C1336850
C0562689
C0108929
C0108929
C0123916
C0123916
C0009564
C0009564
C0009564
C0009271
C0006642
C1339242
C1339242
C1333146
C1333146
C1333146
C1333146
C0083734
C0008878
C1366364
C1366364
C0216098
C0216098
C0034367
C0034367
C0679607
C0296241
C0296241
C0000258
C0000478
C1504335
C0005728
C0870397
C0000396
C1366998
C1366998
C1411634
C1411634
C0757015
C0757015
C1510638
C0259307
C0219844
C0057599
C0057599
C0740275
C0599540
C0001137
C0001903
C0002995
C1124995
C1124995
C0870718
C1456669
C0291165
C0031719
C0031719
C0080098
C0442028
C1579323
C1562128
C

C0382012
C0382012
C0003551
C0003551
C1512770
C0857639
C0857699
C0748408
C0599195
C0175442
C0857120
C0598656
C0302296
C0103376
C0014923
C0333919
C0014923
C1366244
C1412248
C1412248
C0004860
C0238400
C1160602
C1369113
C1369113
C1504541
C1167668
C0492899
C0028514
C0028514
C1589399
C0232838
C0748449
C0242079
C0010777
C0040116
C0948120
C0745178
C0852847
C102723407
C102723407
C102723407
C0848320
C1142028
C102723407
C102723407
C102723407
C1362638
C1362638
C0000213
C0005981
C0079734
C1547020
C0843874
C0871478
C1545922
C0856964
C1292532
C0101991
C0101991
C0854419
C0856759
C0443766
C0210076
C0210076
C0210076
C0210076
C0871342
C0229683
C0858090
C0719508
C1096297
C0262857
C0154853
C0459012
C1262304
C0999183
C0360596
C1322855
C0741299
C0871800
C0332862
C0064084
C0064084
C0064084
C0650607
C0650607
C0000431
C0259266
C0259266
C0259266
C0259266
C0259266
C0259266
C0259266
C0259266
C0237274
C0369671
C0919721
C1112262
C1138832
C0164071
C0164071
C0000431
C0005573
C0005573
C1168027
C0183727
C0561758
C026400

...

C100507195
C100507195
C0257007
C0257007
C5392497
C5392453
C100132354
C100132354
C5241853
C2265061
C2265061
C1340152
C1340152
C2599488
C1140459
C1140459
C0293054
C0293054
C1410535
C1410535
C100288152
C100288152
C5259718
C1411016
C1411016
C1589848
C1589848
C1390212
C1390212
C3177146
C0111345
C0111345
C4068740
C4046503
C4046503
C1151230
C1151230
C1151230
C1151230
C1693218
C1693218
C0849421
C5262713
C3853287
C5398043
C5396237
C5238034
C101929423
C101929423
C1790966
C1790966
C1964053
C1342926
C1342926
C5380740
C2069658
C5238085
C1161502
C1161502
C2911117
C1639220
C100302145
C100302145
C4035206
C1427639
C1427639
C5237206
C5392302
C3533479
C3533479
C3533479
C3533479
C5238199
C5391949
C1855911
C1855911
C5392709
C1400892
C1400892
C2730130
C2730130
C5238052
C5392584
C101928120
C1526224
C1526224
C1256380
C1256380
C2401145
C2401145
C1128710
C1128710
C3842809
C5383286
C1703066
C1703066
C2242496
C2210851
C5397596
C3841865
C1410352
C1410352
C4256598
C4256598
C1539318
C1539318
C2186679
C1426727
C14267

C1112767
C5301272
C1857494
C1857494
C5236065
C3840959
C3840959
C1707929
C1707929
C3655682
C5252132
C3845750
C5384646
C1429842
C1429842
C0205338
C0205338
C5237357
C3509021
C1693211
C1693211
C1979930
C1348093
C1348093
C2120379
C2120379
C5256790
C5253899
C5239846
C0000602
C100874207
C100874207
C0013163
C0013163
C4484139
C1405753
C1405753
C0011035
C0398720
C0398720
C5263071
C5251134
C1857716
C1857716
C100126311
C100126311
C0948225
C4068818
C5240605
C1146862
C1146862
C5241161
C1536192
C1536192
C1427343
C1427343
C5239045
C3840750
C1749987
C1749987
C1749987
C1126731
C1126731
C1484701
C1484701
C5392229
C1117286
C1117286
C1855717
C1855717
C1410325
C1410325
C2229794
C5238371
C1403778
C3140683
C3140683
C1612348
C1612348
C1113451
C1113451
C1426701
C1426701
C5379790
C5374225
C100271873
C100271873
C1339122
C1339122
C100302133
C100302133
C5237717
C5237663
C0878641
C4699987
C1535998
C100302257
C100302257
C5262235
C3647833
C4697835
C2939069
C1440925
C5247533
C100616247
C100616247
C1440435
C1440435
C539

C2042133
C1619279
C1619279
C2026623
C1440186
C1411253
C1411253
C3841725
C4256598
C4256598
C3404201
C3404201
C100313892
C100313892
C2736854
C4729126
C4729126
C0369959
C0369959
C1410300
C1410300
C5380636
C5240344
C4699429
C0007516
C3843687
C1864921
C1864921
C5238145
C1583893
C1583893
C1857616
C1857616
C5396948
C1161291
C1161291
C3534440
C0979794
C0979794
C2894946
C2128452
C5396026
C1701915
C1701915
C2600032
C5392411
C1440590
C1440590
C1457471
C1457471
C1507204
C5392982
C1128077
C1128077
C1148252
C1148252
C4300277
C2169638
C1323047
C1323047
C100616142
C100616142
C5338032
C2005166
C2005166
C100862696
C100862696
C103724390
C103724390
C1581929
C1581929
C1427564
C1427564
C2964213
C5392966
C3401014
C3401014
C5240140
C1403314
C1403314
C100134259
C100134259
C1457864
C1457864
C5392933
C5392253
C2158763
C2158763
C1124404
C1124404
C1456850
C1456850
C3152475
C3152475
C4298905
C0127184
C0127184
C4484277
C100996511
C100996511
C1354476
C1354476
C100313772
C100313772
C1411257
C1411257
C5389883
C1164153


C1525917
C1525917
C5420377
C1699178
C5436988
C5433459
C1654909
C101926941
C101926941
C5459272
C5459272
C5391534
C5237326
C5400042
C5400042
C5438994
C5416819
C5440607
C5424010
C1767566
C1767566
C5420222
C5392620
C5421805
C1418693
C1418693
C1879050
C1879050
C5238957
C3841491
C5432240
C4263167
C5432320
C100505741
C100505741
C100505741
C5435160
C5420653
C5419153
C2223148
C2223148
C2051855
C3282997
C3282997
C0870895
C5421795
C5435177
C5421857
C5421353
C1504508
C1785444
C1785444
C1785444
C5400079
C5419000
C2065876
C5267282
C0146890
C0146890
C5421804
C5421804
C5437568
C2401152
C2401152
C5401386
C3841924
C1391824
C2034491
C5432290
C3846421
C1374403
C1374403
C1374403
C1374403
C5439537
C5438238
C5441082
C5401611
C0651496
C0651496
C1411496
C1411496
C5420337
C5420149
C5433528
C3655564
C4284684
C4284684
C2910232
C1579612
C1579612
C5420379
C5260450
C4068941
C1979780
C5267530
C100500835
C100500835
C5420133
C5421109
C3475490
C5384259
C1855028
C1855028
C1353132
C1353132
C0024594
C0024594
C5420530
C5418

C0871389
C1419488
C1419488
C1584759
C1584759
C100302138
C100302138
C100302138
C2229974
C2229974
C5235619
C100128590
C100128590
C0510988
C0510988
C1510207
C1510207
C2291807
C2291807
C1389860
C1389860
C2727936
C2727936
C2055847
C2055847
C1171483
C1171483
C1457563
C1457563
C0855383
C100302113
C100302113
C1811179
C1811179
C100505495
C100505495
C0399978
C0399978
C4237135
C100874024
C100874024
C1337880
C1337880
C1609527
C3650669
C3650669
C100302250
C100302250
C3856729
C3856729
C1112259
C1140825
C1140825
C1407046
C1407046
C1407046
C0806150
C4067899
C0369077
C1539823
C1539823
C103752588
C103752588
C0920135
C0089876
C0219103
C0219103
C100506428
C1407012
C1407012
C0254773
C104472717
C100507651
C1112797
C0284835
C105375014
C0117824
C0527791
C1457089
C1457089
C0051449
C0254559
C100874203
C1440387
C1440387
C100128327
C1729582
C1729582
C0295347
C0295347
C1148350
C0009258
C0055589
C0001955
C1112491
C0373856
C100132594
C0129450
C0400680
C100131320
C0755595
C0755595
C1149619
C100507032
C0113802
C000270

In [144]:
df[df['ENTITY']=='100616102']

,ENTITY,SEMTYPE,NAME


In [21]:
ent_descriptions

,ENTITY,DEFINITION,ALIASES,NAME
0,C0003725,Arthropod-borne viruses. A non-taxonomic desig...,Arbovirus (navigational concept)|arbovirus|Arb...,Arboviruses
1,C0039258,None,Tahyna virus (organism)|Tahyna virus,Tahyna virus
2,C0318627,None,Eyach virus|Eyach virus (organism),Eyach virus
3,C0012634,A definite pathologic process with a character...,Disease|Clinical disease or syndrome|Clinical ...,Disease
4,C0042776,Minute infectious agents whose genomes are com...,"Virus|Virus, NOS|Viruses, General|Virus (organ...",Virus
...,...,...,...,...
474867,C1644132,None,,Acontia
474868,C0597918,None,American Filipinos|americans filipino|Filipino...,Filipino Americans
474869,C0022038,None,Ioxaglate (substance)|Ioxaglate|Product contai...,ioxaglate
474870,C1289860,None,Flashlamp pulsed dye laser|Flashlamp-pumped pu...,Flashlamp-pumped pulsed dye laser device


In [22]:
ent_descriptions[ent_descriptions['NAME']=='COVID-19']

,ENTITY,DEFINITION,ALIASES,NAME
358902,C5203670,None,Disease caused by 2019-nCoV|Disease caused by ...,COVID-19


In [46]:
for i in list(ent_descriptions['NAME']):
    if isinstance(i, str):
        if 'cov' in i.lower():
            print(i)

Covers (device)
Recovery - action
leucovorin
leucovorin
Circovirus Infections
Helicoverpa armigera
Recovery of Function
Fluorouracil/Irinotecan/Leucovorin Calcium regimen (IFL)
Covered Stent
Vesicovaginal Fistula
Oncovin
Cytoxan-Oncovin-Procarbazine-Prednisone Regimen
leucovorin calcium
leucovorin
leucovorin calcium
Porcine circovirus type II
guided discovery
Fluorescence Recovery After Photobleaching
Roscovitine
media coverage
Oncovin-Prednisone-Procarbazine-Adriamycin Regimen (OPPA Regimen)
Helicoverpa zea
Ice Cover
ancova
Sea Ice Cover
Circovirus
Canary circovirus
Pigeon circovirus
Roscovitine
Incidental Discovery
covalence
Nitrospira moscoviensis
Drug Discovery
Circoviridae
Covalent Interaction
Ascoviridae
leucovorin calcium
total recovery
R-roscovitine
Porcine circovirus 1
Helicoverpa armigera nucleopolyhedrovirus
Escovopsis
Helicoverpa assulta
Muscovy duck parvovirus
Helicoverpa gelotopoeon
Corticoviridae
covenant
Delayed surgical recovery
Porcine circovirus Ag
Porcine circovirus

In [38]:
ent_descriptions[(ent_descriptions['NAME']==None)&(ent_descriptions['DEFINITION']==None)&(ent_descriptions['ALIASES']==None)]

,ENTITY,DEFINITION,ALIASES,NAME


In [12]:
linker.umls.cui_to_entity['C0012634']

CUI: C0012634, Name: Disease
Definition: A definite pathologic process with a characteristic set of signs and symptoms. It may affect the whole body or any of its parts, and its etiology, pathology, and prognosis may be known or unknown.
TUI(s): T047
Aliases (abbreviated, total: 26): 
	 Disease, Clinical disease or syndrome, Clinical disease or syndrome present, NOS, disorder, Disorders, Disease or syndrome present, disease, Diseases and Disorders, Clinical disease AND/OR syndrome present, condition

In [14]:
print(linker.umls.cui_to_entity['C0012634'].definition)

A definite pathologic process with a characteristic set of signs and symptoms. It may affect the whole body or any of its parts, and its etiology, pathology, and prognosis may be known or unknown.


In [16]:
print(linker.umls.cui_to_entity['C0012634'].aliases)

['Disease', 'Clinical disease or syndrome', 'Clinical disease or syndrome present, NOS', 'disorder', 'Disorders', 'Disease or syndrome present', 'disease', 'Diseases and Disorders', 'Clinical disease AND/OR syndrome present', 'condition', 'Diseases', 'Disease or syndrome present, NOS', 'Disorder, NOS', 'Other Disease', 'Clinical disease or syndrome, NOS', 'Disease AND/OR syndrome present', 'Disease or Disorder, Non-Neoplastic', 'Disease (disorder)', 'disorders', 'Clinical disease AND/OR syndrome', 'Disorder', 'Disease or Disorder', 'Clinical disease or syndrome present', 'diseases', 'disease/disorder', 'Disease, NOS']


In [23]:
#dir(linker.umls.cui_to_entity['C0012634'].canonical_name)

In [20]:
linker.umls.cui_to_entity['C0012634'].canonical_name

'Disease'

In [ ]:
linker.umls.cui_to_entity[cui]

In [25]:
ent_descriptions.to_csv(join(DATA_SOURCE, 'entity_description_can.csv'), compression = 'gzip', index = False)

## Entity Descriptions using UMLS Metathe

In [2]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *

### DB Creation

In [3]:
default_world.set_backend(filename = "pym.sqlite3")

In [4]:
import_umls(join(META_DIR, "umls-2022AA-metathesaurus.zip"), terminologies = ["ICD10", "SNOMEDCT_US", "CUI"])

Importing UMLS from /home/pc/Downloads/umls-2022AA-full/2022AA-full/umls-2022AA-metathesaurus.zip with Python version 3.8.12 and Owlready version 2-0.38...
  Parsing umls-2022AA-metathesaurus/2022AA/MRSTY.RRF as MRSTY
  Parsing umls-2022AA-metathesaurus/2022AA/MRRANK.RRF as MRRANK
  Parsing umls-2022AA-metathesaurus/2022AA/MRCONSO.RRF as MRCONSO
  Parsing umls-2022AA-metathesaurus/2022AA/MRDEF.RRF as MRDEF
  Parsing umls-2022AA-metathesaurus/2022AA/MRREL.RRF as MRREL
  Parsing umls-2022AA-metathesaurus/2022AA/MRSAT.RRF as MRSAT
Breaking ORIG cycles...
    SNOMEDCT_US : 0 cycles found: 
    ICD10 : 0 cycles found: 
    SRC : 0 cycles found: 
Finalizing only properties and restrictions...
Finalizing CUI - ORIG mapping...
FTS Indexing...


get_ontology("http://PYM/")

### Concept Extraction

In [6]:
PYM = get_ontology("http://PYM/").load()
CUI = PYM["CUI"]

In [7]:
concept = CUI["C0085580"]

In [9]:
dir(concept)

['INDIRECT_get_properties',
 '__children',
 '__class__',
 '__classcell__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__label',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parents',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__terminology',
 '__weakref__',
 '_equivalent_to',
 '_get_instance_possible_relations',
 '_get_is_instance_of',
 '_instance_equivalent_to_changed',
 '_instance_is_a_changed',
 '_name',
 '_set_is_instance_of',
 'differents',
 'generate_default_name',
 'get_equivalent_to',
 'get_inverse_properties',
 'get_iri',
 'get_name',
 'get_properties',
 'iri',
 'is_a',
 'is_instance_of',
 'name',
 'namespace',
 'set_equivalent_to',
 'set_iri',
 'set_name',
 'storid']

In [22]:
concept.label

['Essential hypertension']

In [23]:
concept.synonyms

['Essential (primary) hypertension',
 'Idiopathic hypertension',
 'Primary hypertension',
 'Systemic primary arterial hypertension',
 'Essential hypertension (disorder)',
 'Essential hypertension']

In [42]:
concept.originals[0].terminology.get_class_properties()

{rdf-schema.label, PYM.unifieds, PYM.synonyms, PYM.term_type, PYM.terminology}

In [46]:
concept.originals[0].terminology.unifieds

[CUI["C1137110"] # ICD10]

In [39]:
concept.originals[0].get_class_properties()

{rdf-schema.label, PYM.unifieds, PYM.synonyms, PYM.term_type, PYM.terminology}

In [30]:
concept.originals[1].get_class_properties()

{PYM.interprets,
 PYM.active,
 PYM.ctv3id,
 PYM.definition_status_id,
 PYM.groups,
 PYM.subset_member,
 rdf-schema.label,
 PYM.type_id,
 PYM.mapped_to,
 PYM.case_significance_id,
 PYM.has_finding_site,
 PYM.unifieds,
 PYM.synonyms,
 PYM.term_type,
 PYM.effective_time,
 PYM.has_interpretation,
 PYM.terminology}

In [51]:
concept.originals[1].subset_member

['900000000000509007~ACCEPTABILITYID~900000000000548007',
 '900000000000508004~ACCEPTABILITYID~900000000000548007',
 '900000000000497000~MAPTARGET~XE0Uc',
 '447562003~MAPCATEGORYID~447637006',
 '447562003~MAPTARGET~I10',
 '447562003~MAPRULE~TRUE',
 '447562003~MAPGROUP~1',
 '447562003~MAPPRIORITY~1',
 '447562003~CORRELATIONID~447561005',
 '6011000124106~MAPCATEGORYID~447637006',
 '6011000124106~MAPRULE~IFA 445518008 &#x7C; Age at onset of clinical finding (observable entity) &#x7C; < 29.0 days',
 '6011000124106~MAPCATEGORYID~447639009',
 '6011000124106~MAPTARGET~I10',
 '6011000124106~MAPADVICE~IF AGE AT ONSET OF CLINICAL FINDING BEFORE 29.0 DAYS CHOOSE P29.2 &#x7C; DESCENDANTS NOT EXHAUSTIVELY MAPPED &#x7C; MAP OF SOURCE CONCEPT IS CONTEXT DEPENDENT',
 '6011000124106~MAPRULE~OTHERWISE TRUE',
 '6011000124106~MAPGROUP~1',
 '6011000124106~MAPPRIORITY~1',
 '6011000124106~MAPPRIORITY~2',
 '6011000124106~CORRELATIONID~447561005',
 '6011000124106~MAPTARGET~P29.2',
 '447562003~MAPADVICE~ALWAYS 

In [38]:
concept.originals[1].definition_status_id

['900000000000074008']

In [53]:
dir(concept)

['INDIRECT_get_properties',
 '__children',
 '__class__',
 '__classcell__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__label',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parents',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__synonyms',
 '__terminology',
 '__weakref__',
 '_equivalent_to',
 '_get_instance_possible_relations',
 '_get_is_instance_of',
 '_instance_equivalent_to_changed',
 '_instance_is_a_changed',
 '_name',
 '_set_is_instance_of',
 'differents',
 'generate_default_name',
 'get_equivalent_to',
 'get_inverse_properties',
 'get_iri',
 'get_name',
 'get_properties',
 'iri',
 'is_a',
 'is_instance_of',
 'name',
 'namespace',
 'set_equivalent_to',
 'set_iri',
 'set_name',
 'storid']

In [61]:
concept.description

AttributeError: 'description' property is not defined.

In [29]:
dir(concept.originals[1])

['INDIRECT_get_properties',
 '__children',
 '__class__',
 '__classcell__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__label',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parents',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__terminology',
 '__weakref__',
 '_equivalent_to',
 '_get_instance_possible_relations',
 '_get_is_instance_of',
 '_instance_equivalent_to_changed',
 '_instance_is_a_changed',
 '_name',
 '_set_is_instance_of',
 'differents',
 'generate_default_name',
 'get_equivalent_to',
 'get_inverse_properties',
 'get_iri',
 'get_name',
 'get_properties',
 'iri',
 'is_a',
 'is_instance_of',
 'name',
 'namespace',
 'set_equivalent_to',
 'set_iri',
 'set_name',
 'storid']

In [24]:
concept.get_class_properties()

{PYM.terminology, PYM.synonyms, rdf-schema.label, PYM.originals}

## Relation Description and Entity Type Description: From UMLS Semantic Network

In [164]:
def get_entity_type_description(df_ent):
    all_data = []
    for i, rec in df_ent.iterrows():
        rec_data = {}
        rec_data['name'] = rec['semantic_type_name/relation_name']
        rec_data['definition'] = rec['definition']
        rec_data['usage_note'] = rec['usage_note']
        rec_data['abbreviation'] = rec['abbreviation']
        all_data.append(rec_data)
    return pd.DataFrame(all_data)

def get_rel_description(df_rel):
    all_data = []
    for i, rec in df_rel.iterrows():
        rec_data = {}
        rec_data['name'] = rec['semantic_type_name/relation_name'].upper()
        rec_data['definition'] = rec['definition']
        rec_data['usage_note'] = rec['usage_note']
        rec_data['relation_inverse'] = rec['relation_inverse'].upper()
        rec_data['abbreviation'] = rec['abbreviation']
        all_data.append(rec_data)
    return pd.DataFrame(all_data)

def get_entity_relation_type_descriptions():
    df = pd.read_csv(join(UMLS_Sem_Net_DIR, 'global_SRDEF.csv'))
    ent_types_df = df[df['record_type'] == "STY"]
    rel_types_df = df[df['record_type'] == "RL"]
    ent_types_df = get_entity_type_description(ent_types_df)
    rel_types_df = get_rel_description(rel_types_df)
    return ent_types_df, rel_types_df


In [165]:
out = get_entity_relation_type_descriptions()

In [166]:
out[0].to_csv(join(DATA_SOURCE,'entity_type_description.csv'), compression = 'gzip', index = False)

In [167]:
out[1].to_csv(join(DATA_SOURCE,'relation_type_description.csv'), compression = 'gzip', index = False)

In [168]:
# pd.read_csv(join(UMLS_Sem_Net_DIR, 'global_SRDEF.csv'))

In [169]:
# pd.read_csv(join(UMLS_Sem_Net_DIR, 'global_SRSTRE2.csv'))

## Tests

In [2]:
!pip install umls-api

  Created wheel for umls-api: filename=umls_api-0.1.0-py3-none-any.whl size=3408 sha256=1099020a5bb8670fbaa5b60128c76d967f1a15499339cee1fdf1158b93987bbe
  Stored in directory: /home/pc/.cache/pip/wheels/70/3f/e4/440933c7b4bf51b0a0efb890e7f8bf01e2530bc80533499e16
Successfully built umls-api


### Via API

In [4]:
import umls_api

In [1]:
UMLS_API_KEY = '770aa413-6b37-4b27-b4ba-5f93464392f1'

In [10]:
umls_inst = umls_api.API(api_key=UMLS_API_KEY)

In [11]:
resp = umls_inst.get_cui(cui='C0007107')

In [12]:
resp

{'pageSize': 25,
 'pageNumber': 1,
 'pageCount': 1,
 'result': {'ui': 'C0007107',
  'name': 'Malignant neoplasm of larynx',
  'dateAdded': '09-30-1990',
  'majorRevisionDate': '08-05-2021',
  'classType': 'Concept',
  'suppressible': False,
  'status': 'R',
  'semanticTypes': [{'name': 'Neoplastic Process',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2022AA/TUI/T191'}],
  'atoms': 'https://uts-ws.nlm.nih.gov/rest/content/2022AA/CUI/C0007107/atoms',
  'definitions': 'https://uts-ws.nlm.nih.gov/rest/content/2022AA/CUI/C0007107/definitions',
  'relations': 'https://uts-ws.nlm.nih.gov/rest/content/2022AA/CUI/C0007107/relations',
  'defaultPreferredAtom': 'https://uts-ws.nlm.nih.gov/rest/content/2022AA/CUI/C0007107/atoms/preferred',
  'atomCount': 250,
  'cvMemberCount': 0,
  'attributeCount': 0,
  'relationCount': 17}}